<a href="https://colab.research.google.com/github/amehregan25/Lead-Assistant/blob/main/marketing_app_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# --- Repo setup ---
REPO_URL = "https://github.com/amehregan25/Lead-Assistant.git"
REPO_NAME = "Lead-Assistant"

# Fresh clone every session so reps always get latest version
!rm -rf {REPO_NAME}
!git clone {REPO_URL}

%cd {REPO_NAME}

# Install any Python deps (ipywidgets at minimum)
%pip install ipywidgets --quiet

import importlib
import interactive_marketing
importlib.reload(interactive_marketing)

print("Repo cloned and interactive_marketing imported.")

Cloning into 'Lead-Assistant'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 32 (delta 12), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 994.20 KiB | 2.72 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/Lead-Assistant/Lead-Assistant/Lead-Assistant/Lead-Assistant
Repo cloned and interactive_marketing imported.


In [8]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import textwrap

def render_report(report: dict):
    clear_output(wait=True)
    lines = []

    lines.append(f"### Recommendation Summary\n")
    lines.append(f"- **Segment**: {report.get('segment', 'N/A')}")
    lines.append(f"- **Persona**: {report.get('persona', 'N/A')}")
    lines.append(f"- **Workflow**: {report.get('topic', 'generic')}")

    install_info = report.get("install_base_info")
    if install_info:
        lines.append("\n#### Current Customer")
        acc = install_info.get("account_name", "Unknown account")
        loc = install_info.get("location", "Unknown location")
        lines.append(f"- Existing **eProtein Discovery** system at **{acc}** ({loc})")
        if install_info.get("note"):
            lines.append(f"- Note: {install_info['note']}")

    cases = report.get("recommended_cases", [])
    if cases:
        lines.append("\n#### Recommended Case Studies")
        for case in cases:
            name = case.get("name", "Untitled case")
            summary = case.get("summary", "").strip()
            lines.append(f"- **{name}**")
            if summary:
                wrapped = textwrap.fill(summary, width=90)
                lines.append(f"  - {wrapped}")
    else:
        lines.append("\n#### Recommended Case Studies")
        lines.append("- No direct case-study overlap; share general workflow resources.")

    resources = report.get("suggested_resources", [])
    if resources:
        lines.append("\n#### Suggested Collateral")
        for r in resources:
            lines.append(f"- {r}")

    history = report.get("campaign_history", [])
    if history:
        lines.append("\n#### Prior Campaign Membership")
        for item in history:
            cname = item.get("campaign_name", "Unknown campaign")
            date = item.get("first_associated_date", "N/A")
            lines.append(f"- {cname} (first associated: {date})")

    display(Markdown("\n".join(lines)))


# Widgets
first_name_w = widgets.Text(description="First name", layout=widgets.Layout(width="45%"))
last_name_w = widgets.Text(description="Last name", layout=widgets.Layout(width="45%"))

title_w = widgets.Text(
    description="Title",
    placeholder="e.g. Principal Scientist, Protein Engineering",
    layout=widgets.Layout(width="90%"),
)

company_w = widgets.Text(
    description="Company",
    placeholder="e.g. Pfizer",
    layout=widgets.Layout(width="90%"),
)

workflow_w = widgets.Dropdown(
    description="Workflow",
    options=[
        ("Auto / generic", ""),
        ("Soluble", "soluble"),
        ("Membrane", "membrane"),
        ("Antibody", "antibody"),
    ],
    value="",
    layout=widgets.Layout(width="60%"),
)

protein_targets_w = widgets.Text(
    description="Protein(s)",
    placeholder="e.g. GIPR, GCGR, FFAR1",
    layout=widgets.Layout(width="90%"),
)

therapeutic_areas_w = widgets.Text(
    description="Therapeutic areas",
    placeholder="e.g. oncology, metabolic disease, neurodegeneration",
    layout=widgets.Layout(width="90%"),
)

linkedin_summary_w = widgets.Textarea(
    description="LinkedIn summary",
    placeholder="Paste lead's LinkedIn summary (optional)…",
    layout=widgets.Layout(width="90%", height="120px"),
)

run_button = widgets.Button(
    description="Generate recommendations",
    button_style="primary",
    layout=widgets.Layout(width="60%"),
)

output_area = widgets.Output()

def on_run_clicked(b):
    with output_area:
        output_area.clear_output()

        report = interactive_marketing.generate_recommendation_report(
            title=title_w.value.strip(),
            company=company_w.value.strip(),
            topic_input=workflow_w.value or "",
            protein_targets=protein_targets_w.value.strip(),
            therapeutic_areas=therapeutic_areas_w.value.strip(),
            linkedin_summary=linkedin_summary_w.value.strip(),
            first_name=first_name_w.value.strip(),
            last_name=last_name_w.value.strip(),
        )
        render_report(report)

run_button.on_click(on_run_clicked)

form = widgets.VBox([
    widgets.HTML("<h3>Nuclera Case Study Recommender</h3>"),
    widgets.HBox([first_name_w, last_name_w]),
    title_w,
    company_w,
    workflow_w,
    protein_targets_w,
    therapeutic_areas_w,
    linkedin_summary_w,
    run_button,
])

app = widgets.VBox([form, output_area])
display(app)